In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 90% 73.0M/80.9M [00:00<00:00, 111MB/s] 
100% 80.9M/80.9M [00:00<00:00, 91.2MB/s]


In [ ]:
# extracting compressed file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk #nltk= natural language toolkit
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#printing the stopwords in english- these words dont affect the tweets daata
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# **data processing**

In [ ]:


twitter_dataset = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')


In [ ]:
#checking no. of rows and columns
twitter_dataset.shape

(1599999, 6)

In [ ]:
#pprinting first 5 rows
twitter_dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#name the columns and read again
column_names=['target','ids','date','flag','user','text']
twitter_dataset = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')

In [ ]:
twitter_dataset.shape

(1600000, 6)

In [ ]:
twitter_dataset.head()


,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#counting the no. of missing values
twitter_dataset.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0
stemmed_content,0


In [ ]:
#checking the distribution of target columns
twitter_dataset['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
#convert the target '4'to '1'
twitter_dataset.replace({'target': {4:1}}, inplace=True)

In [ ]:
#checking the distribution of target columns
twitter_dataset['target'].value_counts()

,count
target,
0,800000
1,800000


0--> Negative
1---> positive

# **STEMMING**
It is the process of reducing a word to its root word

eg- actors,actress,actring- root word=act

In [ ]:
 port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z ]','',content)  # Keep spaces
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content




In [ ]:
twitter_dataset['stemmed_content'] = twitter_dataset['text'].apply(stemming)

In [ ]:
twitter_dataset.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot httptwitpiccomyzl awww that bummer ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset cant updat facebook text might cri resul...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav im mad cant see


In [ ]:
print(twitter_dataset['stemmed_content'])

0          switchfoot httptwitpiccomyzl awww that bummer ...
1          upset cant updat facebook text might cri resul...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                      nationwideclass behav im mad cant see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdbcom cool hear old walt interview httpbli...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_dataset['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#sepearatring data and label
X = twitter_dataset['stemmed_content'].values
Y = twitter_dataset['target'].values

In [ ]:
print(X)

['switchfoot httptwitpiccomyzl awww that bummer shoulda got david carr third day'
 'upset cant updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguphh']


In [ ]:
 print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)


In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin im'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day cant wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [ ]:
print(X_test)

['mmangen fine havent much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [ ]:
#converting actual data to numerical data
#vectorizer-asigns importance to each word on basis of repititon

vectorizer = TfidfVectorizer()


X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

  (0, 590524)	0.271935973896376
  (0, 485829)	0.35734734000828555
  (0, 275153)	0.5287163126006141
  (0, 135672)	0.3751210798745192
  (0, 330512)	0.4200753866446622
  (0, 599548)	0.4490363130629349
  (1, 199672)	0.9736525845879415
  (1, 265990)	0.2280364982304841
  (2, 135672)	0.46222653304912487
  (2, 265990)	0.11789035957962253
  (2, 153180)	0.1905056115706456
  (2, 552451)	0.1883319097978993
  (2, 160229)	0.29161611256480313
  (2, 551043)	0.3225385174402054
  (2, 586742)	0.3320039094486021
  (2, 96518)	0.3154389006079266
  (2, 600077)	0.33687662733057383
  (2, 368923)	0.24369409966007974
  (2, 555076)	0.1531113773352699
  (2, 186397)	0.1889237729533197
  (2, 163730)	0.20357992328484822
  (2, 395952)	0.16862711364982916
  (3, 551043)	0.28972941659572027
  (3, 197116)	0.4472150500940576
  (3, 188008)	0.2779040561459071
  :	:
  (1279996, 587294)	0.27112935728623483
  (1279996, 531652)	0.22006519052096826
  (1279996, 509536)	0.3605270658820192
  (1279996, 334168)	0.5223794704929838
  (1

In [ ]:
print(X_test)

  (0, 18735)	0.17227271385168627
  (0, 39020)	0.15827445867713713
  (0, 84315)	0.2612587662159671
  (0, 131471)	0.35874567833011956
  (0, 163803)	0.24913841224739333
  (0, 171233)	0.23191862402987573
  (0, 199992)	0.2179563421051184
  (0, 258522)	0.27403042037763914
  (0, 374099)	0.440155313081504
  (0, 384931)	0.17345900492444394
  (0, 529152)	0.21456939414403495
  (0, 541819)	0.3387821615303071
  (0, 555076)	0.30718915166161176
  (0, 571715)	0.17566897667440953
  (1, 8008)	0.3053245680397504
  (1, 18735)	0.2144214773747336
  (1, 180088)	0.5700107365388162
  (1, 310429)	0.40607312375462945
  (1, 356802)	0.28423752828699317
  (1, 478367)	0.4807076997545237
  (1, 500333)	0.24314985399219477
  (2, 28150)	0.35334726711330017
  (2, 43324)	0.3799460828604977
  (2, 111216)	0.36316906406311267
  (2, 272498)	0.5865060038453336
  :	:
  (319994, 600596)	0.25994636406247995
  (319995, 133815)	0.37415114246535175
  (319995, 135835)	0.3387823265608489
  (319995, 192780)	0.31055261383164634
  (31999

TRAINING THE ML MODEL

LOGISTIC REGRESSION

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
MODEL EVALUATION

In [ ]:
ACCURACY SCORE

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.80700703125


In [ ]:
#accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.781696875


Model accuracy=78.16%

Saving trained model

In [ ]:
import pickle

In [ ]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

Using Model for future predictions

In [ ]:
#loading model
loaded_model=pickle.load(open('trained_model.sav','rb'))

In [ ]:
X_new=X_test[200]
print(Y_test[200])
prediction=loaded_model.predict(X_new)
print(prediction)

if prediction[0]==0:
  print('Negative Tweet')
else:
  print('Positive Tweet')

1
[1]
Positive Tweet


In [ ]:
X_new=X_test[3]
print(Y_test[3])
prediction=loaded_model.predict(X_new)
print(prediction)

if prediction[0]==0:
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[0]
Negative Tweet


In [ ]:
new_tweet = input("Enter your tweet: ")
processed_tweet = stemming(new_tweet)
transformed_tweet = vectorizer.transform([processed_tweet])
prediction = loaded_model.predict(transformed_tweet)

Enter your tweet: my sisters are the worst


In [ ]:
if prediction[0] == 0:
    print('Negative Tweet')
else:
    print('Positive Tweet')


Negative Tweet
